In [ ]:
import osmnx as ox
from ipyleaflet import *
import networkx as nx
import plotly.graph_objects as obj
import numpy as np

##  creating graph using OSMnx

In [ ]:
G = ox.graph_from_place('university of toronto')
fig, ax = ox.plot_graph(G)

In [ ]:
# converting a graph to node and/or edge GeoDataFrames.
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
# rendering graph nodes
nodes

In [ ]:
# rendering graph edges
edges

In [ ]:
# rendering the 2nd node 
list(G.nodes(data=True))[1]

In [ ]:
# rendering the 1std edge
list(G.edges(data=True))[1]

In [ ]:
# Converting node and edge GeoDataFrames to a MultiDiGraph.
G2=ox.graph_from_gdfs(nodes, edges)
ig2, ax2 = ox.plot_graph(G2)

In [ ]:
# graph stats
ox.basic_stats(G)

## plotting the map usinng folium and save the graph as html file

In [ ]:
ox.plot_graph_folium(G)

## plotting map using ipyleaflet

In [ ]:
center=(43.662643, -79.395689) # UofT main building
source_point = (43.664527, -79.392442)  # King Edward VII Equestrian Statue
destination_point = (43.659659, -79.397669) # Bahen Centre for Information Technology at UofT

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=False)

# King Edward VII Equestrian Statue
marker1 = Marker(name='marker1', location=source_point)

# Bahen Centre for Information Technology
marker2 = Marker(name='marker2', location=destination_point)
m.add_layer(marker1)
m.add_layer(marker2)

control = LayersControl(position='topright')
m.add_control(control)
m.add_layer(marker);
m

In [ ]:
# getting the nearest nodes to the locations 
source_node = ox.get_nearest_node(G, source_point) 
destination_node = ox.get_nearest_node(G, destination_point)
# Render the closest node id to origin and destination points 
source_node, destination_node

## working with networkx

In [ ]:
# finding the shortest path usuing dijkstra algorithm
route = nx.shortest_path(G, source_node, destination_node, weight = 'length') 

In [ ]:
# getting latitude and longitude of the nodes in the shortest path
longitude = [] 
latitude = []  
for i in route:
     point = G.nodes[i]
     longitude.append(point['x'])
     latitude.append(point['y'])

## plotting the shortest route usning plotly

In [ ]:
fig = obj.Figure(obj.Scattermapbox(name = "Path", mode = "lines", lat = latitude, lon = longitude, marker = {'size': 10}, line = dict(width = 4.5, color = 'blue')))

# source node
fig.add_trace(obj.Scattermapbox(name = "Source", mode = "markers", lat = [source_point[0]], lon = [source_point[1]], 
                                marker = {'size': 12, 'color':"red"}))
     
# destination node
fig.add_trace(obj.Scattermapbox(name = "Destination", mode = "markers", lat = [destination_point[0]], lon = [destination_point[1]], 
                                marker = {'size': 12, 'color':'green'}))
    
# center for plots
lat_center = np.mean(latitude)
long_center = np.mean(longitude)

# defining plot layout
fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, mapbox = {'center': {'lat': lat_center, 'lon': long_center}, 'zoom': 13})
fig.show()

## finding point of interest (POI) using overpass-api

In [ ]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(node["amenity"="restaurant"](around:1000,43.66, -79.39);
 way["amenity"="restaurant"](around:1000,43.66, -79.39);
 rel["amenity"="restaurant"](around:1000,43.66, -79.39);

);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
restaurant = response.json()

In [ ]:
len(restaurant['elements'])

In [ ]:
# Extract the lon, lat and name of each restaurant:
lons = []
lats = []
text = []
for elem in restaurant['elements']:
    if elem['type'] == 'node':
        lons.append(elem['lon'])
        lats.append(elem['lat'])
        
    elif 'center' in elem:
        lons.append(elem['center']['lon'])
        lats.append(elem['center']['lat'])  
    else: pass
    if 'name'  not in elem['tags']:
        text.append('Noname')
    else:
        text.append(elem['tags']['name'])

In [ ]:
restaurants = dict(type='scattermapbox',
                   lat=lats, 
                   lon=lons,
                   mode='markers',
                   text=text,
                   marker=dict(size=8, color='blue'),
                   hoverinfo='text',    
                   showlegend=False)

In [ ]:
# plotting restaurants' location around University of Toronto
fig = obj.Figure(obj.Scattermapbox(restaurants))

# defining plot layout
fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, mapbox = {'center': {'lat': lat_center, 'lon': long_center}, 'zoom': 13})
fig.show()